# Automatic Short Answer Grading

Calificación de una base de preguntas dataset de semeval'13


In [153]:
# Libraries
import random
import sys
import xml.etree.ElementTree as ET
#import sswe
import pandas as pd
import numpy as np
import keras
import theano
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cross_validation import train_test_split
from scipy.spatial import distance
from sklearn.metrics import accuracy_score


# Text
import gensim, logging
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import f1_score as f1
from sklearn import cross_validation
import re
from nltk.util import ngrams
from collections import *
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.models import Doc2Vec


# Clasifiers
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.linear_model import SGDClassifier


# LSTM 
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing import sequence

# xml reader
import glob
import os

In [3]:
_fname = "/home/aecarrillor/ASAG/GoogleNews-vectors-negative300.bin"
w2vModel = Doc2Vec.load_word2vec_format(_fname, binary=True)
stp_wrds=stopwords.words('english')## Caching the stopwords object. calling in tokenizer function makes a bottleneck.

In [218]:
def txt_data(text,remove_stpwrds=False):
    nst=tknz_sntnc(text,remove_stpwrds)
    text_len=len(nst)
    if len(nst)==0:
        text_len=0.0001
    wrd_in_w2vec=sum([1  if word in w2vModel else 0.0 for word in nst])
    match_prop=wrd_in_w2vec/float(text_len)
    return [text_len,wrd_in_w2vec,match_prop]

def tknz_sntnc(text, remove_stpwrds):
    tokenizer = RegexpTokenizer('\w+\'+\w+|\w+')#r'\w+'
    tokens=tokenizer.tokenize(text)
    tokens=[wrd.lower()for wrd in tokens]
    if remove_stpwrds:#==True:
        filtered_words = [word for word in tokens if word not in stp_wrds]#stp_wrds must be declared as a global object
        return filtered_words
    else:
        return tokens
    # Meter stopwords removal
    
def tkns2vec(tokens, normalize):
    if len(tokens)==0:
        return np.array([0 for i in xrange(300)])# Si el tokenizador da un vector vacío, se corrige on un vector de ceros
    ar_w2v=[w2vModel[word] if word in w2vModel else np.array([0 for i in xrange(300)]) for word in tokens]# si la palabra no está en el embedding, arroja un vector de ceros.
    #ar_w2v=[w2vModel[word] if word in w2vModel else np.array(["""""""" for i in xrange(300)]) for word in tokens]# si la palabra no está en el embedding, arroja un vector de numeros aleatorios con seed en la palabra ??? cómo obtener el seed.
    if normalize==True:
        return sum(ar_w2v)/len(ar_w2v)
    else:
        return sum(ar_w2v)
    #OOV_words=["" if word in w2vModel else word for word in tokens]

def w2vec_txt(text, normalize, remove_stpwrds):
    tokens=tknz_sntnc(text,remove_stpwrds)
    return tkns2vec(tokens, normalize)
    

In [214]:
def xml_2_900_vec(path_files, normalize=False, remove_stpwrds=False):

    #Variables internas
    all_files =  glob.glob(path_files+'*.xml')

    #Procesos

    ## crear array con id de pregunta, textos combinados y calificación.
    id_question=[]# Identificador de la pregunta (etiqueta de clase).
    calif=[]# Marca de criterio.
    quest=[]#Pregunta.
    ref_answer=[]#Respuesta de referencia.
    stud_answ=[]#Respuesta de estudiante.


    for doc in all_files:
        tree = ET.parse(doc)
        root = tree.getroot()
        question=root[0].text
        grade_st=[branch.attrib["accuracy"] for branch in root[2]]
        answers_st=[branch.text for branch in root[2]]
        answers_ref=[branch.text for branch in root[1]]# If there are more of 1 reference answer...
        #print (doc, root.attrib["id"])
        for ith_ans,st_answ in enumerate(answers_st):
            quest.append(question)
            ref_answer.append(answers_ref[0])
            stud_answ.append(st_answ)
            id_question.append(root.attrib["id"])
            st_ans_calif=str(grade_st[ith_ans])
    #             calif.extend([st_ans_calif]) # Categorías en texto para clasificadores multiclase
            calif.extend([1 if st_ans_calif=="correct" else 0])
            #print(ith_ans)
    arr_in=np.column_stack((id_question,quest,ref_answer,stud_answ))
    arr_w2v=[[w2vec_txt(text,normalize, remove_stpwrds) for text in ans[1:]]for ans in arr_in]
    vec_fin=[list(arr2v[0])+list(arr2v[1])+list(arr2v[2])for arr2v in arr_w2v]
    return [vec_fin,calif,arr_in]

## 2-way 

###ScientisBank


In [241]:
nrmalz=True
rmv_stp=False

stud_answ=xml_2_900_vec('S13T7Data/2and3way/training/2way/sciEntsBank/',nrmalz,rmv_stp)
X_train_w2v=stud_answ[0]
y_train_w2v=stud_answ[1]
print ("Training:",len(y_train_w2v))

unseen_ans=xml_2_900_vec('S13T7Data/2and3way/test/2way/sciEntsBank/test-unseen-answers/',nrmalz,rmv_stp)
X_unseen_ans_w2v=unseen_ans[0]
y_unseen_ans_w2v=unseen_ans[1]
print ("UA:",len(y_unseen_ans_w2v))

unseen_doms=xml_2_900_vec('S13T7Data/2and3way/test/2way/sciEntsBank/test-unseen-domains/',nrmalz,rmv_stp)
X_unseen_doms_w2v=unseen_doms[0]
y_unseen_doms_w2v=unseen_doms[1]
print ("UD:",len(y_unseen_doms_w2v))

unseen_qstns=xml_2_900_vec('S13T7Data/2and3way/test/2way/sciEntsBank/test-unseen-questions/',nrmalz,rmv_stp)
X_unseen_qstns_w2v=unseen_qstns[0]
y_unseen_qstns_w2v=unseen_qstns[1]
print ("UQ:",len(y_unseen_qstns_w2v))

Training: 4969
UA: 540
UD: 4562
UQ: 733


In [240]:
data=np.array(stud_answ[2])[:,1:]
OOV_stud_ans_stop_prop=np.array([[txt_data(txt,True) for txt in answrs]for answrs in data])[:,:,2]
props_OOV_arr=np.array(OOV_stud_ans_stop_prop)
props_OOV_arr
np.mean(props_OOV_arr)

#plt.hist(OOV_stud_ans_stop_prop)

0.97943544733564303

In [242]:
stpwinW2vec=[1 if stp in w2vModel else 0 for stp in stp_wrds]
np.mean(stpwinW2vec)
stpwNotInW2vec=[stp for stp in stp_wrds if stp not in w2vModel ]
#word for word in tokens if word not in stp_wrds
stpwNotInW2vec
#w2vModel["a"]

[u'a', u'and', u'of', u'to', u'mightn', u'mustn']

In [148]:
# Create classifiers
lr = LogisticRegression()
gnb = GaussianNB()
svc = SGDClassifier(loss="hinge", penalty="l2", n_iter=1500)# probar con otras funciones perdida # Mean Square Error
rfc = RandomForestClassifier(n_estimators=100)

f1_UA=[]
f1_UD=[]
f1_UQ=[]

clasifs_w2v=[]

for clf, name in [(lr, 'Logistic'),
                  (gnb, 'Naive Bayes'),
                  (svc, 'SVM -Lineal'),
                  (rfc, 'Random Forest')]:
    clf.fit(X_train_w2v, y_train_w2v)
    yhat_UA = clf.predict(X_unseen_ans_w2v)
    yhat_UD = clf.predict(X_unseen_doms_w2v)
    yhat_UQ = clf.predict(X_unseen_qstns_w2v)
    f1_UA.append(f1(y_unseen_ans_w2v, yhat_UA, average='weighted'))####### F1 weighted Average parameter
    f1_UD.append(f1(y_unseen_doms_w2v, yhat_UD, average='weighted'))####### F1 weighted Average parameter
    f1_UQ.append(f1(y_unseen_qstns_w2v, yhat_UQ, average='weighted'))####### F1 weighted Average parameter
    clasifs_w2v.append(name)

results_w2v=pd.DataFrame(np.column_stack(([round(scr,4) for scr in f1_UA],[round(scr,4) for scr in f1_UQ],[round(scr,4) for scr in f1_UD])),
                         columns=["UA","UQ", "UD"], index=clasifs_w2v)
results_w2v

,UA,UQ,UD
Logistic,0.5509,0.4615,0.2517
Naive Bayes,0.4989,0.5928,0.4871
SVM -Lineal,0.5347,0.5094,0.3962
Random Forest,0.6699,0.2027,0.0460


### MLP
- se hace fuera del loop de los clasifacdores para poder explorar parámetros.

http://scikit-learn.org/stable/modules/neural_networks_supervised.html

In [248]:
from sklearn.neural_network import MLPClassifier

ImportError: cannot import name MLPClassifier

In [247]:
clf = MLPClassifier(solver='lbfgs', alpha=0.1,
                    hidden_layer_sizes=(300,60,1), random_state=1)
clf.fit(X_train_w2v, y_train_w2v)

yhat_UA_MLP = clf.predict(X_unseen_ans_w2v)
yhat_UD_MLP = clf.predict(X_unseen_doms_w2v)
yhat_UQ_MLP = clf.predict(X_unseen_qstns_w2v)


NameError: name 'MLPClassifier' is not defined

### Bettle

In [144]:
stud_answ=xml_2_900_vec('S13T7Data/2and3way/training/2way/beetle/',nrmalz,rmv_stp)
X_train_w2v=stud_answ[0]
y_train_w2v=stud_answ[1]

unseen_ans=xml_2_900_vec('S13T7Data/2and3way/test/2way/beetle/test-unseen-answers/',nrmalz,rmv_stp)
X_unseen_ans_w2v=unseen_ans[0]
y_unseen_ans_w2v=unseen_ans[1]

#This dataset doesn't have Unknown Domain
# unseen_doms=xml_2_900_vec('S13T7Data/2and3way/test/2way/beetle/test-unseen-domains/',nrmalz,rmv_stp)
# X_unseen_doms_w2v=unseen_doms[0]
# y_unseen_doms_w2v=unseen_doms[1]

unseen_qstns=xml_2_900_vec('S13T7Data/2and3way/test/2way/beetle/test-unseen-questions/',nrmalz,rmv_stp)
X_unseen_qstns_w2v=unseen_qstns[0]
y_unseen_qstns_w2v=unseen_qstns[1]

In [145]:
# Create classifiers
lr = LogisticRegression()
gnb = GaussianNB()
svc = SGDClassifier(loss="hinge", penalty="l2", n_iter=1500)
rfc = RandomForestClassifier(n_estimators=100)

f1_UA=[]
#f1_UD=[]
f1_UQ=[]

clasifs_w2v=[]

for clf, name in [(lr, 'Logistic'),
                  (gnb, 'Naive Bayes'),
                  (svc, 'SVM -Lineal'),
                  (rfc, 'Random Forest')]:
    clf.fit(X_train_w2v, y_train_w2v)
    yhat_UA = clf.predict(X_unseen_ans_w2v)
    #yhat_UD = clf.predict(X_unseen_doms_w2v)
    yhat_UQ = clf.predict(X_unseen_qstns_w2v)
    f1_UA.append(f1(y_unseen_ans_w2v, yhat_UA, average='weighted'))####### F1 weighted Average parameter
    #f1_UD.append(f1(y_unseen_doms_w2v, yhat_UD, average='weighted'))####### F1 weighted Average parameter
    f1_UQ.append(f1(y_unseen_qstns_w2v, yhat_UQ, average='weighted'))####### F1 weighted Average parameter
    clasifs_w2v.append(name)

results_w2v=pd.DataFrame(np.column_stack(([round(scr,4) for scr in f1_UA],[round(scr,4) for scr in f1_UQ])),
                         columns=["UA", "UQ"], index=clasifs_w2v)
results_w2v

,UA,UQ
Logistic,0.6023,0.5258
Naive Bayes,0.5836,0.4804
SVM -Lineal,0.6018,0.5260
Random Forest,0.7688,0.5374
